In [ ]:
##MLP model 

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np

#x is input(total_flux) and y is output(flow)

# Prepare the data
# Split the data into training, validation, and testing sets (60:20:20)
X_train, X_temp, y_train, y_temp = train_test_split(x, y, test_size=0.4, random_state=42) # 60% train, 40% temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 20% val, 20% test


# Convert data to PyTorch tensors

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


# Define the neural network architecture

class Net(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3,output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out

# Hyperparameters

input_size = 2*len(edge_list) 
hidden_size1 = 128  
hidden_size2 = 64
hidden_size3 = 32
output_size = len(edge_list)
learning_rate = 0.0001
num_epochs = 10000

# Initialize the model, loss function, and optimizer
model = Net(input_size, hidden_size1,hidden_size2,hidden_size3, output_size)

if num_trips >1:
    criterion = nn.MSELoss()  # Use Mean Squared Error for regression
else:
    criterion = nn.BCELoss()  # Use Binary Cross Entropy Loss

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
losses =[]
val_losses = []
best_val_loss = float('inf')

# Training loop with validation
for epoch in range(num_epochs): 
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    if epoch == 1234:
        print(outputs)
        print(y_train)
    losses.append(loss.item())

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)
        val_losses.append(val_loss.item())
        if (epoch+1)%10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss.item():.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth') # Save the best model
            ctr=ctr+1


    if (epoch+1)%10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Load the best model
model.load_state_dict(torch.load('best_model.pth'))



# Evaluation
with torch.no_grad():
    predicted = model(X_test)
    test_loss = criterion(predicted, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')


In [ ]:
#loss curve for training and validation for 100,1000 and 10000 epochs

import matplotlib.pyplot as plt

for n in range(2,5):

    # Plotting the test loss curve
    plt.plot(losses[:pow(10,n)])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss Curve')
    plt.grid(True)
    
    plt.show()
    
    # Plotting the valuation loss curve
    plt.plot(val_losses[:pow(10,n)])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Validation Loss Curve')
    plt.grid(True)
    plt.show()

In [ ]:
#plot histogram/frequency analysis for each dataset 

import torch
import matplotlib.pyplot as plt
def create_sidebyside_histogram(data1,data2):
    
    # Define bin size and range
    bin_size = 4
    min_value = 0
    max_value = 40
    
    # Create the histograms using torch.histc
    hist1 = torch.histc(data1, bins=(max_value // bin_size), min=min_value, max=max_value)
    hist2 = torch.histc(data2, bins=(max_value // bin_size), min=min_value, max=max_value)
    
    # Plotting the histograms side by side using subplots
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns
    
    # Plot for data1
    axs[0].bar(range(0, max_value, bin_size), hist1.numpy(), width=bin_size, align='edge', edgecolor='black')
    axs[0].set_title('Histogram for Data 1')
    axs[0].set_xlabel('Value')
    axs[0].set_ylabel('Frequency')
    axs[0].set_xticks(range(0, max_value + 1, bin_size))
    
    # Plot for data2
    axs[1].bar(range(0, max_value, bin_size), hist2.numpy(), width=bin_size, align='edge', edgecolor='black')
    axs[1].set_title('Histogram for Data 2')
    axs[1].set_xlabel('Value')
    axs[1].set_ylabel('Frequency')
    axs[1].set_xticks(range(0, max_value + 1, bin_size))
    
    # Show the plots
    plt.tight_layout()
    plt.show()




In [ ]:
j = 0
for i in range(20):
    j = random.randint(1,50)
    # Convert tensors to numpy arrays for plotting
    predicted_np = predicted[j].numpy().flatten()
    # print(predicted_np)
    # y_test is already a numpy array so, .numpy() call is removed
    actual_np = y_test[j].flatten()
    # print(actual_np)
    create_sidebyside_histogram(predicted[j],y_test[j])
    
    

In [ ]:
# prompt: plot the predictions of above neural network v/s the expected output on test dataset
import numpy as np
import matplotlib.pyplot as plt
import random

# Assuming 'predicted' and 'y_test' are available from the previous code
# ... (your existing code)

j = 0
for i in range(20):
    j = random.randint(1,199)
    

    # Convert tensors to numpy arrays for plotting
    predicted_np = predicted[j].numpy().flatten()
    # print(predicted_np)
    # y_test is already a numpy array so, .numpy() call is removed
    actual_np = y_test[j].numpy().flatten()
    # print(actual_np)
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    plt.plot(actual_np, label='Actual')
    plt.plot(predicted_np, label='Predictions')
    plt.xlabel('Edge List')
    plt.ylabel('Flow')  # Assuming you're predicting 'flow'
    plt.title(f'Predicted vs Actual on Test Dataset {j} and Trips {t[j-1]}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
#for each each - mse, weighted mse and mae b/w y_true and y_pred across test dataset

import numpy as np
import matplotlib.pyplot as plt


def mse(y_true, y_pred):
    return np.mean((np.array(y_true) - np.array(y_pred)) ** 2)

def mae

# Assuming 'predicted' and 'y_test' are available from the previous code
#hashmaps key = 'edge' , value = 'statistics = weighted mse,mse and mae' 
dfl={}
dfj={}
dlk={}
for j in range(212):
    predicted_np=[]
    actual_np=[]
    for i in range(200): 
        # Convert tensors to numpy arrays for plotting
        predicted_np.append((predicted[i].numpy().flatten()[j]))
        # print(predicted_np)
        # y_test is already a numpy array so, .numpy() call is removed
        actual_np.append(y_test[i].numpy().flatten()[j])
        # print(actual_np)
        # Create the plot
    
    # print(predicted_np,actual_np)
    # print(mse(predicted_np,actual_np) /pow((max(predicted_np+actual_np)-min(actual_np+predicted_np)),1))
    
    dfl[j] = (mse(predicted_np,actual_np)/pow((max(predicted_np+actual_np)-min(actual_np+predicted_np)),1))
    dfj[j] = mse(predicted_np,actual_np)
    
    print(dfl[j],dfj[j])
    
    plt.figure(figsize=(10, 6))
    plt.plot(actual_np, label='Actual')
    plt.plot(predicted_np, label='Predictions')
    plt.xlabel('Dataset')
    plt.ylabel('Flow')  # Assuming you're predicting 'flow'
    plt.title(f'Flow for edge{j}')
    plt.legend()
    plt.grid(True)
    plt.show()



In [ ]:
import matplotlib.pyplot as plt

print(dfl)

data = dfl

# Extract keys and values
x = list(data.keys())  # Labels (Months)
y = list(data.values())  # Values (Sales)

# Create a figure
plt.figure(figsize=(8, 5))


# Plot as a bar chart
plt.bar(x, y, alpha=0.5, color='c')

# Labels and title
plt.xlabel("Edge")
plt.ylabel("Scaled MSE Loss")
plt.title("Scaled MSE Statistics for each edge")
plt.legend()

# Show the graph
plt.show()